# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL (Quiz)") \
    .getOrCreate()

In [3]:
path = "dataset/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [11]:
# TODO: write your code to answer question 1
spark.sql('''
    SELECT
        distinct page
    FROM
        user_log_table
    WHERE
        page NOT IN (
            SELECT
                distinct page
            FROM 
                user_log_table
            WHERE
                userId = ''
        )        
''').collect()

[Row(page='Submit Downgrade'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [13]:
# TODO: write your code to answer question 3
spark.sql('''
    SELECT
        count(distinct userId) as female_count
    FROM
        user_log_table
    WHERE
        gender = 'F'
''').toPandas()

,female_count
0,462


# Question 4

How many songs were played from the most played artist?

In [18]:
# TODO: write your code to answer question 4
spark.sql('''
    SELECT
        artist,
        count(*) as n
    FROM
        user_log_table
    WHERE
        artist IS NOT NULL
    GROUP BY
        artist
    ORDER BY 2 DESC
    LIMIT 1
''').toPandas()

,artist,n
0,Coldplay,83


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [21]:
spark.udf.register('period', lambda x: int(x.lower() == 'home'), IntegerType())

<function __main__.<lambda>(x)>

In [24]:
spark.sql('''
SELECT
ROUND(AVG(PRD)) avg_prd
FROM (
    SELECT 
        userId, 
        page, 
        count(nt) PRD 
    FROM
    (
          SELECT userId, page, ts, period(page), 
          SUM(period(page)) OVER(PARTITION BY userId ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as nt
          FROM user_log_table 
          WHERE
              page in ('Home', 'NextSong')
    ) as t
    GROUP BY
        userId,
        page
) AS F
          '''
).toPandas()

,avg_prd
0,7.0
